In [2]:
#SVM  DONE!!
import numpy as np
import pandas as pd
from keras import layers
from sklearn import svm
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,roc_auc_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="white")
#sns.set(style=whitegrid, color_codes=True)

data = pd.read_excel(r'Dropbox/Project personal(400)/DATA FOR COMPUTER SCIENCE STUDENT NEW.xls',sheet_name='INACTIVE EMPLOYEES',  parse_date={'Expire Date'})

#eliminating attributes not needed
data = data.drop(['Personnel Number','Country Name','Birth Date','Age.1'],axis=1)
print(data.shape)
print(data.head())
data.dtypes

Using TensorFlow backend.


(4329, 9)
  Gnd Marital Status  Age Ethnic Group Name           Post Name  \
0   M              S   60     GREATER ACCRA     SENIOR LECTURER   
1   M              S   61           CENTRAL     SENIOR LECTURER   
2   M              M   53           ASHANTI     SENIOR LECTURER   
3   F              S   39           EASTERN            LECTURER   
4   M              S   36           EASTERN  PART-TIME LECTURER   

               Appt. Type                 Department Name Expire Date  \
0  SENIOR MEMBER ACADEMIC                MODERN LANGUAGES  2005-08-01   
1  SENIOR MEMBER ACADEMIC                MODERN LANGUAGES  1997-10-01   
2  SENIOR MEMBER ACADEMIC  DEPT. OF NUTRITION & FOOD SCI.  2016-01-01   
3  SENIOR MEMBER ACADEMIC    POP.FAMILY & REPR.HEALTH-SPH  2006-08-01   
4  SENIOR MEMBER ACADEMIC                           ISSER  2002-08-01   

  Reason for Separation  
0            RETIREMENT  
1            RETIREMENT  
2  VOLUNTARY RETIREMENT  
3              RESIGNED  
4      VACATION OF

Gnd                              object
Marital Status                   object
Age                               int64
Ethnic Group Name                object
Post Name                        object
Appt. Type                       object
Department Name                  object
Expire Date              datetime64[ns]
Reason for Separation            object
dtype: object

In [3]:
#finding the correlation
data.count()
data.groupby('Reason for Separation').mean()
#from the meann grouping below, it is evident that most employees retired due to their age
#followed by those who requested for an early retirement before voluntary retirement
#conclusion: this shows that most employees from the University retired as compared to the other factors under reason of termination.

,Age
Reason for Separation,
DECEASED,49.220339
DISMISSED,36.051282
EARLY RETIREMENT,56.000000
END OF APPOINTMENT,41.657895
NON-CONFIRMATION,42.800000
RESIGNED,35.477573
RETIREMENT,60.241641
SEPARATION,36.300000
TERMINATED,40.125000


In [4]:
#finding the correlation
data.count()
data.groupby('Marital Status').mean()

,Age
Marital Status,
D,54.562500
M,53.957550
S,37.164889
W,55.000000


In [5]:
data['Expire Date'] = pd.to_datetime(data['Expire Date'])
data['year'], data['month'],data['day'] = data['Expire Date'].dt.year, data['Expire Date'].dt.month, data['Expire Date'].dt.day
data.head()

,Gnd,Marital Status,Age,Ethnic Group Name,Post Name,Appt. Type,Department Name,Expire Date,Reason for Separation,year,month,day
0,M,S,60,GREATER ACCRA,SENIOR LECTURER,SENIOR MEMBER ACADEMIC,MODERN LANGUAGES,2005-08-01,RETIREMENT,2005,8,1
1,M,S,61,CENTRAL,SENIOR LECTURER,SENIOR MEMBER ACADEMIC,MODERN LANGUAGES,1997-10-01,RETIREMENT,1997,10,1
2,M,M,53,ASHANTI,SENIOR LECTURER,SENIOR MEMBER ACADEMIC,DEPT. OF NUTRITION & FOOD SCI.,2016-01-01,VOLUNTARY RETIREMENT,2016,1,1
3,F,S,39,EASTERN,LECTURER,SENIOR MEMBER ACADEMIC,POP.FAMILY & REPR.HEALTH-SPH,2006-08-01,RESIGNED,2006,8,1
4,M,S,36,EASTERN,PART-TIME LECTURER,SENIOR MEMBER ACADEMIC,ISSER,2002-08-01,VACATION OF POST,2002,8,1


In [6]:
data = data.drop(['Expire Date'], axis=1)
#data.dtypes['Expire Date'] = pd.to_datetime(data['Expire Date'])
data.dtypes 

Gnd                      object
Marital Status           object
Age                       int64
Ethnic Group Name        object
Post Name                object
Appt. Type               object
Department Name          object
Reason for Separation    object
year                      int64
month                     int64
day                       int64
dtype: object

In [7]:
#data['Expire Data'].astype(np.int64) = data['Expire Data']
numeric_data = list(data.dtypes[data.dtypes != "object"].index)
X1 = data[numeric_data]
print(X1.head())
#data.dtypes["Expire Date"] = data.dtypes(Timestamp)

   Age  year  month  day
0   60  2005      8    1
1   61  1997     10    1
2   53  2016      1    1
3   39  2006      8    1
4   36  2002      8    1


In [8]:
data.groupby('Reason for Separation').count()
#from the count results below,  its evident that employees who retired are the most in the dataset , followed by those who resigned.
#vacation of post also gave the third highest number of employees in from the dataset.
# The least number of employees from the reason for separation was for those that were non-confirmed
#All other forms of retirement(early retirement and voluntary retirement) will be classified under retirement.

,Gnd,Marital Status,Age,Ethnic Group Name,Post Name,Appt. Type,Department Name,year,month,day
Reason for Separation,,,,,,,,,,
DECEASED,413,413,413,413,413,413,413,413,413,413
DISMISSED,39,39,39,39,39,39,39,39,39,39
EARLY RETIREMENT,1,1,1,1,1,1,1,1,1,1
END OF APPOINTMENT,76,76,76,76,76,76,76,76,76,76
NON-CONFIRMATION,5,5,5,5,5,5,5,5,5,5
RESIGNED,758,758,758,758,758,758,758,758,758,758
RETIREMENT,1974,1974,1974,1974,1974,1974,1974,1974,1974,1974
SEPARATION,40,40,40,40,40,40,40,40,40,40
TERMINATED,160,160,160,160,160,160,160,160,160,160


In [9]:
#Classification of Reason for Separation
data['Reason for Separation']=np.where(data['Reason for Separation']=='EARLY RETIREMENT','RETIREMENT',data['Reason for Separation'])
data['Reason for Separation']=np.where(data['Reason for Separation']=='VOLUNTARY RETIREMENT','RETIREMENT',data['Reason for Separation'])
data['Reason for Separation']=np.where(data['Reason for Separation']=='SEPARATION','RESIGNED',data['Reason for Separation'])
data['Reason for Separation']=np.where(data['Reason for Separation']=='VACATION OF POST','RESIGNED',data['Reason for Separation'])
data['Reason for Separation']=np.where(data['Reason for Separation']=='DISMISSED','TERMINATED',data['Reason for Separation'])
data['Reason for Separation']=np.where(data['Reason for Separation']=='NON-CONFIRMATION','TERMINATED',data['Reason for Separation'])
data['Reason for Separation']=np.where(data['Reason for Separation']=='DECEASED','TERMINATED',data['Reason for Separation'])
data['Reason for Separation']=np.where(data['Reason for Separation']=='END OF APPOINTMENT','TERMINATED',data['Reason for Separation'])
data.groupby('Reason for Separation').count()



,Gnd,Marital Status,Age,Ethnic Group Name,Post Name,Appt. Type,Department Name,year,month,day
Reason for Separation,,,,,,,,,,
RESIGNED,1459,1459,1459,1459,1459,1459,1459,1459,1459,1459
RETIREMENT,2177,2177,2177,2177,2177,2177,2177,2177,2177,2177
TERMINATED,693,693,693,693,693,693,693,693,693,693


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4329 entries, 0 to 4328
Data columns (total 11 columns):
Gnd                      4329 non-null object
Marital Status           4329 non-null object
Age                      4329 non-null int64
Ethnic Group Name        4329 non-null object
Post Name                4329 non-null object
Appt. Type               4329 non-null object
Department Name          4329 non-null object
Reason for Separation    4329 non-null object
year                     4329 non-null int64
month                    4329 non-null int64
day                      4329 non-null int64
dtypes: int64(4), object(7)
memory usage: 372.1+ KB


In [11]:
#isolating the target variable
Y_raw = data.pop("Reason for Separation")
#creating dummmy variables of target
Y = pd.get_dummies(Y_raw)
print(Y.head())
#info()

   RESIGNED  RETIREMENT  TERMINATED
0         0           1           0
1         0           1           0
2         0           1           0
3         1           0           0
4         1           0           0


In [12]:
X1_train, X1_test, Y_train, Y_test = train_test_split(X1, Y, test_size = 0.2,random_state = 0)
model = OneVsRestClassifier(estimator = SVC(gamma = 'scale', random_state = 0, kernel ='linear', C= 1))#this is used because the dataset has many classes which are interrelated 
#hence its a multiclass
model.fit(X1_train, Y_train)
model_predictions = model.predict(X1_test)

count_misclassified = (Y_test != model_predictions).sum()
print('Misclassified samples: {}'.format(count_misclassified))
print(classification_report(Y_test,model_predictions))
print("Accuracy : ", accuracy_score(Y_test,model_predictions))
print ("c-stat:", roc_auc_score(Y_test,model_predictions))
print(confusion_matrix(Y_test.values.argmax(axis=1),model_predictions.argmax(axis=1)))

Misclassified samples: RESIGNED       95
RETIREMENT     65
TERMINATED    139
dtype: int64
              precision    recall  f1-score   support

           0       0.82      0.87      0.84       295
           1       0.90      0.95      0.93       432
           2       0.00      0.00      0.00       139

   micro avg       0.87      0.77      0.82       866
   macro avg       0.57      0.61      0.59       866
weighted avg       0.73      0.77      0.75       866
 samples avg       0.77      0.77      0.77       866

Accuracy :  0.7713625866050808
c-stat: 0.7699566871270201
[[287   8   0]
 [ 20 412   0]
 [102  37   0]]


C:\Users\Anna\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Anna\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Anna\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Anna\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predi

In [13]:
categorical_data = list(data.dtypes[data.dtypes == "object"].index)
#creating dummy variables.
X2 = pd.get_dummies(data[categorical_data])
print(X2.head())


   Gnd_F  Gnd_M  Marital Status_D  Marital Status_M  Marital Status_S  \
0      0      1                 0                 0                 1   
1      0      1                 0                 0                 1   
2      0      1                 0                 1                 0   
3      1      0                 0                 0                 1   
4      0      1                 0                 0                 1   

   Marital Status_W  Ethnic Group Name_AMERICAS  Ethnic Group Name_ASHANTI  \
0                 0                           0                          0   
1                 0                           0                          0   
2                 0                           0                          1   
3                 0                           0                          0   
4                 0                           0                          0   

   Ethnic Group Name_ASIA  Ethnic Group Name_BRONG AHAFO  ...  \
0                       0  

In [14]:
X = pd.concat([X1,X2],axis = 1)
X.head()

,Age,year,month,day,Gnd_F,Gnd_M,Marital Status_D,Marital Status_M,Marital Status_S,Marital Status_W,...,Department Name_UNIVERSITY CONSULTANCY CENTRE,Department Name_UNIVERSITY HOSPITAL,Department Name_VC'S OFFICE,Department Name_VICE-CHANCELLORS GHANA,Department Name_VIROLOGY UNIT,Department Name_VOLTA HALL,Department Name_VRBRP,Department Name_WACCI,Department Name_WATER DRAINAGE & SEW. SECTION,Department Name_WORKS SECTION
0,60,2005,8,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,61,1997,10,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,53,2016,1,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,39,2006,8,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,36,2002,8,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2,random_state = 0)
model = OneVsRestClassifier(estimator = SVC(gamma = 'scale', random_state = 0, kernel ='linear', C= 1))#this is used because the dataset has many classes which are interrelated 
#hence its a multiclass

model.fit(X_train, Y_train)
model_predictions = model.predict(X_test)
model_mse = mean_squared_error(Y_test,model_predictions)
print(model_mse)
count_misclassified = (Y_test != model_predictions).sum()
print('Misclassified samples: {}'.format(count_misclassified))
print(classification_report(Y_test,model_predictions))
print("Accuracy : ", accuracy_score(Y_test,model_predictions))
print ("c-stat:", roc_auc_score(Y_test,model_predictions))
print(confusion_matrix(Y_test.values.argmax(axis=1),model_predictions.argmax(axis=1)))

0.12240184757505773
Misclassified samples: RESIGNED       99
RETIREMENT     70
TERMINATED    149
dtype: int64
              precision    recall  f1-score   support

           0       0.83      0.84      0.83       295
           1       0.89      0.95      0.92       432
           2       0.35      0.09      0.14       139

   micro avg       0.85      0.77      0.81       866
   macro avg       0.69      0.63      0.63       866
weighted avg       0.78      0.77      0.77       866
 samples avg       0.76      0.77      0.77       866

Accuracy :  0.7528868360277137
c-stat: 0.7740274858674182
[[279  11   5]
 [ 19 411   2]
 [ 93  38   8]]


C:\Users\Anna\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
